In [1]:
import cv2
import torch
import time
import numpy as np

In [2]:
# Load a MiDas model for depth estimation 
#model_type = "DPT_Large"
#model_type = "DPT_Hybrid"
model_type = "MiDaS_small" 

midas = torch.hub.load("intel-isl/MiDaS", model_type)
# Move model to GPU if available
device = torch.device("mps") if torch.cuda.is_available() else torch.device("cpu")
midas.to(device)
midas.eval()

Using cache found in /Users/muratguc/.cache/torch/hub/intel-isl_MiDaS_master


Loading weights:  None


Using cache found in /Users/muratguc/.cache/torch/hub/rwightman_gen-efficientnet-pytorch_master


MidasNet_small(
  (pretrained): Module(
    (layer1): Sequential(
      (0): Conv2dSameExport(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
      (3): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): ReLU6(inplace=True)
          (se): Identity()
          (conv_pw): Conv2d(32, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act2): Identity()
        )
      )
      (4): Sequential(
        (0): InvertedResidual(
          (conv_pw): Conv2d(24, 144, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(144,

In [3]:
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

Using cache found in /Users/muratguc/.cache/torch/hub/intel-isl_MiDaS_master


In [4]:
if model_type == "DPT_Large" or model_type == "DPT_Hybrid":
    transform = midas_transforms.dpt_transform
else:
    transform = midas_transforms.small_transform

In [ ]:
cap = cv2.VideoCapture(1)

In [ ]:
while cap.isOpened():
    success, img = cap.read()
    
    start = time.time()
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    input_batch = transform(img).to(device)
    
    # Prediction and resize to original resolution
    with torch.no_grad():
        prediction = midas(input_batch)
        
        prediction = torch.nn.functional.interpolate(
            prediction.unsqueeze(1),
            size=img.shape[: 2],
            mode="bicubic", 
            align_corners=False,
        ).squeeze()
    
    depth_map = prediction.cpu().numpy()
    depth_map = cv2.normalize(depth_map, None, 0, 1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_64F)

    end = time.time()
    totalTime = end - start
    
    fps = 1 / totalTime
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    depth_map = (depth_map*255). astype(np.uint8)
    depth_map = cv2.applyColorMap(depth_map, cv2.COLORMAP_MAGMA)
    
    cv2.putText(img, f'FPS: {int(fps)}', (20,70), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0,255,0), 2)
    cv2.imshow('Image', img)
    cv2.imshow('Depth Map', depth_map)
    if cv2.waitKey (5) & 0b11111111 == 27:
        break

cap.release()
    

In [ ]:
input_image_path = "/Users/muratguc/Projects/SeniorProject/ObjectClassification/test/downstair.jpg"
input_image = cv2.imread(input_image_path)
input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)

input_image = transform(input_image).to(device)

with torch.no_grad():
    depth_prediction = midas(input_image)

depth_map = depth_prediction.squeeze().cpu().numpy()

depth_map = (depth_map - depth_map.min()) / (depth_map.max() - depth_map.min()) * 255
depth_map = depth_map.astype(np.uint8)

cv2.imshow("Input Image", cv2.cvtColor(cv2.imread(input_image_path), cv2.COLOR_BGR2RGB))
cv2.imshow("Depth Map", depth_map)
cv2.waitKey(0)
cv2.destroyAllWindows()

[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
